In [ ]:
from utils import *
from DataParser import DataParser
from model import MyModel
from predict import predict_cat_dog
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from sklearn.metrics import plot_confusion_matrix
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import shutil
import glob
import cv2
import numpy as np
import zipfile
import json

In [ ]:
#install data and extract

#!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"abdullah0susuz","key":"414ba30eea54d34edb7294627c00488c"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dogs-vs-cats
files = glob.glob('*.zip')
for file in files:
  print(file)
  with zipfile.ZipFile(file, 'r') as zip_ref:
          zip_ref.extractall('PetImages')
!unzip /content/PetImages/train.zip
!unzip /content/PetImages/test1.zip

In [ ]:
#parameteres for model

train_dir = "train"

validation = 0.3
batch_size = 16
image_size = 224
epochs = 20
rate = 1

dataParser = DataParser(train_dir, validation, batch_size, image_size,rate)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
my_model = MyModel((image_size,image_size,3))
model = my_model.build_model(vgg_model=True)
print(model.summary())

In [ ]:
#train model
history = model.fit(generate_batches(dataParser),
          steps_per_epoch=dataParser.steps_per_epoch,
          epochs=epochs,
          validation_data = generate_batches(dataParser, train=False),
          validation_steps=dataParser.validation_steps,
          callbacks=[callback])

with open('file.json', 'w') as f:
    json.dump(history.history, f)

model.save("save-16.h5")

In [ ]:
#plot loss graph

history_dict = json.load(open("file.json", 'r'))
plt.plot(history_dict['val_loss'], label = "val_loss")
plt.plot(history_dict['loss'], label = "loss")
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
#predict all validations
data,label=dataParser.get_batch(dataParser.validation_ids)
y_pred = model.predict(data)>0.5

In [ ]:
#plot and visualize confusion matrix
IC = type('IdentityClassifier', (), {"predict": lambda i : i, "_estimator_type": "classifier"})
plot_confusion_matrix(IC, y_pred, label, normalize='true', values_format='.2%')

In [ ]:
#predict photo in given index
predict_cat_dog(model,number=278,image_size=image_size)